# Кластеризация существительных по левому глагольному контексту
Наша гипотеза состоит в том, что мы можем с помощью алгоритма кластеризации найти группы глаголов, которые ведут себя сходным образом. Для ее проверки мы исследуем существительные в подходящих контекстах: биграммы с переходными глаголами и существительными в соответствующих падежах (кроме именительного и предложного) и триграммы с переходными глаголами, предлогом и существительным.

## Данные
Данные взяты из НКРЯ (поиск по биграммам и триграммам).
### Запрос:
Обращение к данным происходит путем подстановки леммы в заранее сформированный запрос (см. reference.txt), извлечение данных - с помощью XPath.

### Существительные:
В качестве леммы в запросе к корпусу использовались существительные, собранные нами из пособий, а также те слова, которые были получены в результате прошлого интерактива.

### Выход:
Мы получаем данные в виде html-страницы с таблицей, строки которой имеют вид:

In [11]:
import lxml.html
import urllib.request
import urllib.error
import urllib.parse

BIGRAM_LINK =  "http://search.ruscorpora.ru/search.xml?env=sas1_2&mycorp=&mysent=" \
               "&mysize=&mysentsize=&dpp=100&spp=100&spd=100&text=lexgramm&mode=ngrams_2_lexgr&sort=gr_freq&" \
               "lang=ru&nodia=1&parent1=0&level1=0&lex1=&gramm1=V%2Ctran&flags1=&parent2=0&level2=0&min2=1" \
               "&max2=1&lex2={0}&gramm2=%28gen%7Cgen2%7Cdat%7Cacc%7Cacc2%7Cins%29&flags2="
TRIGRAM_LINK = "http://search.ruscorpora.ru/search.xml?env=sas1_2&mycorp=&mysent=" \
               "&mysize=&mysentsize=&dpp=100&spp=100&spd=100&text=lexgramm&mode=ngrams_3_lexgr&sort=gr_freq" \
               "&lang=ru&nodia=1&parent1=0&level1=0&lex1=&gramm1=V&flags1=&parent2=0&level2=0&min2=1" \
               "&max2=1&lex2=&gramm2=PR&flags2=&parent3=0&level3=0&min3=1&max3=1&lex3={0}&gramm3=&flags3="

agent_name = "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36"
            
            
# загрузка страниц
def load_page(url, encoding="cp1251"):
    """
    Простая функция для загрузки страницы с сайта

    :param url: URI of the page
    :param encoding: page encoding
    :return: tuple: a boolean showing success, content of the page (or error message), and http code if available (or 0)
    """
    try:
        req = urllib.request.Request(urllib.parse.quote(url, safe=":/&=?"), headers={'User-Agent': agent_name})
        with urllib.request.urlopen(req) as r:
            code = r.getcode()
            page = r.read().decode(encoding)
            loaded = True
    except urllib.error.HTTPError as e:
        page = e.reason
        code = e.code
        loaded = False
    except urllib.error.URLError as e:
        page = e.reason
        code = 0
        loaded = False
    except Exception as e:
        page = str(e)
        code = 0
        loaded = False
    return loaded, page, code

def load_words(path):
    """
    Загрузка списка слов
    """
    words = list("foo")
    return words

def create_link(word, is_trigram=False):
    """
    Создает ссылки на страницы в соответствии с необходимым запросом
    """
    encoded = urllib.parse.quote(word)
    if is_trigram:
        return TRIGRAM_LINK.format(encoded)
    else:
        return BIGRAM_LINK.format(encoded)

def extract_words_freqs(bi_page, tri_page):
    """
    Извлекает из текста страницы глаголы/глаголы с предлогами и возвращает в виде словаря, где глаголы - ключи, частоты - 
    значения.
    """
    output = dict()
    bi_tree = lxml.html.fromstring(bi_page)
    # обходим таблицу на странице по строкам
    for tr in bi_tree.iter('tr'):
        # прямо в ячейках живут цифры (номер пп. и частотность)
        print(str(tr.xpath(".//td/text()")))
        # в тегах span живут слова
        print(str(tr.xpath(".//td/span/text()")))
    return output

def loader(words):
    output = dict()
    for word in words:
        bi_loaded, bi_page, code = load_page(create_link(word, False))
        tri_loaded, tri_page, code = load_page(create_link(word, True))
        if bi_loaded and tri_loaded:
            output[word] = extract_words_freqs(bi_page, tri_page)
            
        



In [14]:
extract_words_freqs('<tr><td align="right">27</td><td align="right">15</td><td align="right">13</td><td width="100%"> <span class="b-wrd-expl g-em" explain="%D0%BB%D1%8E%D0%B1%D0%B8%D0%BB%D0%B8%7C134%7C10%7C0%7C0%7C0%7C0%7C0">любили</span> <span class="b-wrd-expl g-em" explain="%D0%BE%D1%82%D1%86%D0%B0%7C1087%7C1%7C0%7C0%7C0%7C0%7C0">отца</span> </td></tr>', "")


['27', '15', '13', ' ', ' ', ' ']
['любили', 'отца']


{}